# `generate_labels`
This script generates images to label via Amazon SageMaker

In [1]:
import numpy as np
import imageio
import scipy.io as sio
from scipy.special import comb

from scipy.stats import norm
import matplotlib.pyplot as plt

import keras
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, TensorBoard
from keras.losses import mean_squared_error
from keras.utils import multi_gpu_model
from keras.models import load_model
from keras.layers import Conv2D, Input
from keras.models import Model
from keras.optimizers import Adam

import os
import time
os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.environ['IMAGEIO_FFMPEG_EXE'] = '/home/twd/.local/bin/ffmpeg'

import json

import sys
sys.path.append('../ratception/pose_clusters/')
sys.path.append('../ratception/')

import processing
from processing import plot_markers_3d
import ops
from generator import DataGenerator
from generator_aux import DataGenerator_downsample

from losses import mask_nan_keep_loss, mask_nan_keep_loss_safe, metric_dist_max
import nets


import serve_data
import tensorflow as tf

from six.moves import cPickle

import matplotlib
#matplotlib.use("Agg") #for writing figure videos
%matplotlib notebook
%load_ext autoreload
%autoreload

Using TensorFlow backend.


First, let's load in the mocap data, together with pointers to the correct frames. For now, this is the same across all cameras

In [2]:
RESULTSDIR = './results/calibrd4_50frames_cams356/'
# Create directory if it doesn't already exist

directory = RESULTSDIR
if not os.path.exists(directory):
    os.makedirs(directory)
    
CONFIG_PARAMS = {}

CONFIG_PARAMS['BASEDIR'] = '/media/twd/Fast/data/calib_rd4_hires/'
CONFIG_PARAMS['RESULTSDIR'] = RESULTSDIR
CONFIG_PARAMS['datadir'] = os.path.join(CONFIG_PARAMS['BASEDIR'],'New_preprocessed')
CONFIG_PARAMS['viddir'] = '/media/twd/Fast/data/calib_rd4_hires/Video/'
CONFIG_PARAMS['datafile'] = ['hires1_cam3_RealMatchedFrames.mat',
                             'hires1_cam5_RealMatchedFrames.mat',
                             'hires1_cam6_RealMatchedFrames.mat']
CONFIG_PARAMS['INPUT_WIDTH'] = 1280
CONFIG_PARAMS['INPUT_HEIGHT'] = 1024
CONFIG_PARAMS['OUTPUT_WIDTH'] = 1280
CONFIG_PARAMS['OUTPUT_HEIGHT'] = 1024
CONFIG_PARAMS['CROP_HEIGHT'] = (0,1024)
CONFIG_PARAMS['CROP_WIDTH'] = (20,1300)
CONFIG_PARAMS['N_CHANNELS_IN'] = 3
CONFIG_PARAMS['N_CHANNELS_OUT'] = 20
CONFIG_PARAMS['BBOX_WIDTH'] = 1280
CONFIG_PARAMS['BBOX_HEIGHT'] = 1024

CONFIG_PARAMS['CAMNAMES'] = ['CameraE','CameraS','CameraU2']
CONFIG_PARAMS['BATCH_SIZE'] = 1

CONFIG_PARAMS['WORKERS'] = 12
CONFIG_PARAMS['MAX_QUEUE_SIZE'] = 12

CONFIG_PARAMS['SIGMA'] = 10
  
CONFIG_PARAMS['EPOCHS'] = 5
CONFIG_PARAMS['VERBOSE'] = 1
CONFIG_PARAMS['lr'] = 1e-3
CONFIG_PARAMS['loss'] = mean_squared_error
CONFIG_PARAMS['metric'] = 'mse'#metric_dist_max
CONFIG_PARAMS['net'] = nets.unet2d_fullbn
CONFIG_PARAMS['TILEFAC'] = 1
CONFIG_PARAMS['DOWNFAC'] = 1

CONFIG_PARAMS['IMMODE'] = 'video'

# CONFIG_PARAMS['NEW_N_CHANNELS_OUT'] = 9
# CONFIG_PARAMS['NEW_LAST_KERNEL_SIZE'] = (3,3)

#CONFIG_PARAMS['WEIGHTS'] = '/home/twd/Dropbox/autoencoder/v3.0/results/unet2d_6cams_triangulate/unet2d_6cams/training_weights3_validloss1.05e-4.hdf5'
#write config file
# processing.write_config(RESULTSDIR,CONFIG_PARAMS,
#                         "fine tuning chihuahua 2D w/ the same samples used for 3D fine tuning")

In [3]:
#load data
samples_, datadict_, datadict_3d_, data_3d_ = serve_data.prepare_data(CONFIG_PARAMS, com_flag=False)

Now we need to preload handles to each video file for quick access

In [4]:
vids = {}
vid_dir_flag = True
if CONFIG_PARAMS['IMMODE'] == 'video':
    vids = {}
    for i in range(len(CONFIG_PARAMS['CAMNAMES'])):
        if vid_dir_flag:
            addl = ''
        else:
            addl = os.listdir(os.path.join(CONFIG_PARAMS['viddir'],CONFIG_PARAMS['CAMNAMES'][i]))[0]
        vids[CONFIG_PARAMS['CAMNAMES'][i]] = \
        processing.generate_readers(CONFIG_PARAMS['viddir'],
                                    os.path.join(CONFIG_PARAMS['CAMNAMES'][i],addl),
                                    maxopt=70000, extension='.mp4')

NOTE: Ignoring mp4 files numbered above 70000
NOTE: Ignoring mp4 files numbered above 70000
NOTE: Ignoring mp4 files numbered above 70000


In [5]:
for i in range(len(CONFIG_PARAMS['CAMNAMES'])):
#     if CONFIG_PARAMS['IMMODE'] == 'video':
#         vids[CONFIG_PARAMS['CAMNAMES'][i]] = processing.generate_readers(CONFIG_PARAMS['viddir'],CONFIG_PARAMS['CAMNAMES'][i],
#                                                                         maxopt=70000)
    if CONFIG_PARAMS['IMMODE'] == 'tif':
        vids[CONFIG_PARAMS['CAMNAMES'][i]] = os.path.join(CONFIG_PARAMS['viddir'],CONFIG_PARAMS['CAMNAMES'][i],'tif')

We now set up our generators

In [6]:
# Parameters
#NOTE: SETTING VALIDATION OT RANDOM SHUFFLE
params = {'dim_in': (CONFIG_PARAMS['INPUT_HEIGHT'],CONFIG_PARAMS['INPUT_WIDTH']),
          'n_channels_in': CONFIG_PARAMS['N_CHANNELS_IN'],
          'dim_out': (CONFIG_PARAMS['OUTPUT_HEIGHT'],CONFIG_PARAMS['OUTPUT_WIDTH']),
          'batch_size': CONFIG_PARAMS['BATCH_SIZE'],
          'n_channels_out': CONFIG_PARAMS['N_CHANNELS_OUT'],
          'out_scale': CONFIG_PARAMS['SIGMA'],
          'camnames': CONFIG_PARAMS['CAMNAMES'],
          'crop_width': CONFIG_PARAMS['CROP_WIDTH'],
          'crop_height': CONFIG_PARAMS['CROP_HEIGHT'],
          'bbox_dim': (CONFIG_PARAMS['BBOX_HEIGHT'],CONFIG_PARAMS['BBOX_WIDTH']),
          'downsample': CONFIG_PARAMS['DOWNFAC'],
          'immode': CONFIG_PARAMS['IMMODE'],
          'shuffle': True}

Load in the samples used for 3D chihuahua fine tuning

In [7]:
labels = datadict_
labels_3d = datadict_3d_

In [8]:
partition = {}
partition['train'] = np.random.choice(samples_,size=(50,))

In [9]:
# Generators
train_generator = DataGenerator_downsample(partition['train'], labels, vids, **params)

Initialize the model

In [10]:
train_generator.save_for_dlc(RESULTSDIR + 'imDir/')

/home/twd/anaconda3/envs/keras2.2.2/lib/python3.6/site-packages/scipy/io/matlab/miobase.py:414: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  arr[empties] = ' '
